# Subaru Data Reduction Notebook

## Set the Reduction Parameters

In the box below, you specify various parameters of the reduction process. These include the root directory of the reduction process. Included here is the directory that contains all of the fits files to be used in the reduction.

The process includes only fits files produced using a specified filter. This filter is specified by the variable `filter_name`.

In [1]:
#observation parameters

clean = False # if true, remove all directories and start afresh

filter_name = 'N-A-L671' #process all files for this filter
rootdir = '/home/kevin/Documents/Pelican'
raw_fits_dir = 'all_fits' # where all the raw fits files live

fits_out = 'Pelican_sii.fits' #name of the resulting image which will go rootdir

combined_bias_dir = '/home/kevin/Documents/M8data/M8/combined_bias' # set to None if no bias files

repo_dir = '/home/kevin/repos/ReipurthBallyProject' #directory where the repo was cloned

remove_cosmic_rays = False #True invokes ccdproc.cosmicray_lacosmic (adds 10 minutes to processing)

In [2]:
# static data
import os, sys

sys.path.append(repo_dir)

coord_maps_dir = os.path.join(repo_dir,'SubaruCoordinateMaps')

In [3]:
from pyraf import iraf

from astropy.io import fits
import numpy as np 
import pandas as pd
from ccdproc import ImageFileCollection
from src.pyrafutils import  subaru_reduction, obs_dirs
import shutil


In [5]:
kw = ['FRAMEID', 'EXP-ID', 'DATA-TYP', 'EXPTIME', 'FILTER01', 'DETECTOR']
raw_fits = ImageFileCollection(os.path.join(rootdir, raw_fits_dir), keywords=kw)

#did we get valid filter?
filters = raw_fits.values('FILTER01', unique=True)
if filter_name not in filters:
    print(f'Invalid filter name: {filter_name}')
    print(f'valid filters names are: {filters}')
    raise ValueError()

## Create Coordinate Transformation Maps

In [6]:
image_dir = os.path.join(rootdir, filter_name)
#zap the image directory if needed
if clean:
    #blow it all away
    try:
        shutil.rmtree(image_dir)
    except:
        pass
if not os.path.exists(image_dir):
    os.mkdir(image_dir)

os.chdir(image_dir)
sred = subaru_reduction(filter_name, rootdir)

dirs = obs_dirs(rootdir, filter_name)


In [7]:

try:
    shutil.rmtree(dirs['coord_maps'])
except:
    pass

coord_maps = [p for p in os.listdir(coord_maps_dir) if p.endswith('.coo')]
os.mkdir(dirs['coord_maps'])
for p in coord_maps:
    src = os.path.join(coord_maps_dir, p)
    dst = os.path.join(dirs['coord_maps'], p)
    shutil.copy(src,dst)

detectors = ['chihiro', 'clarisse', 'fio', 'kiki', 'nausicaa', 'ponyo', 'san', 'satsuki', 'sheeta', 'sophie']

for d in detectors:
    res, res_df = sred.map_detector(d, degree=3)


## Remove Bias and Overscan Regions

In [8]:
from src.no_bias import remove_oscan
import warnings

try:
    shutil.rmtree(dirs['no_bias'])
except:
    pass
os.mkdir(dirs['no_bias'])

image_filter = {'DATA-TYP':'OBJECT', 'FILTER01': filter_name}
im_files = raw_fits.files_filtered(include_path=True, **image_filter)

for imf in im_files:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        #need the real header, apparently CCDData.read doesn't return WCS in header
        with fits.open(imf) as hdul:
            hdr = hdul[0].header.copy()
            data = hdul[0].data.astype(np.float32)

        detector = hdr['DETECTOR']
        print(f'file: {os.path.basename(imf)}, detector: {detector}')

        if combined_bias_dir is not None:
            bias_path = os.path.join(combined_bias_dir, detector+ '.fits')
            with fits.open(bias_path) as f:
                bias = f[0].data.copy()
        else:
            bias = None

        new_hdr, no_oscan = remove_oscan(hdr, data, bias)

        phdu = fits.PrimaryHDU(data = no_oscan, header=new_hdr)
        outfile = os.path.join(dirs['no_bias'], os.path.basename(imf))
        phdu.writeto(outfile, overwrite=True)

file: SUPA01469800.fits, detector: nausicaa
file: SUPA01469801.fits, detector: kiki
file: SUPA01469802.fits, detector: fio
file: SUPA01469803.fits, detector: sophie
file: SUPA01469804.fits, detector: sheeta
file: SUPA01469805.fits, detector: satsuki
file: SUPA01469806.fits, detector: chihiro
file: SUPA01469807.fits, detector: clarisse
file: SUPA01469808.fits, detector: ponyo
file: SUPA01469809.fits, detector: san
file: SUPA01469810.fits, detector: nausicaa
file: SUPA01469811.fits, detector: kiki
file: SUPA01469812.fits, detector: fio
file: SUPA01469813.fits, detector: sophie
file: SUPA01469814.fits, detector: sheeta
file: SUPA01469815.fits, detector: satsuki
file: SUPA01469816.fits, detector: chihiro
file: SUPA01469817.fits, detector: clarisse
file: SUPA01469818.fits, detector: ponyo
file: SUPA01469819.fits, detector: san
file: SUPA01469820.fits, detector: nausicaa
file: SUPA01469821.fits, detector: kiki
file: SUPA01469822.fits, detector: fio
file: SUPA01469823.fits, detector: sophie
f

## Register the Images

In [9]:
from ccdproc import ImageFileCollection
try:
    shutil.rmtree(dirs['registered_image'])
except:
    pass
os.mkdir(dirs['registered_image'])

imgs = ImageFileCollection(dirs['no_bias'])

for img in imgs.files:
    print(f'Tranforming: {img}')
    res=sred.transform_image(os.path.splitext(img)[0], remove_cosmic_rays = remove_cosmic_rays)

Tranforming: SUPA01469800.fits
Tranforming: SUPA01469801.fits
Tranforming: SUPA01469802.fits
Tranforming: SUPA01469803.fits
Tranforming: SUPA01469804.fits
Tranforming: SUPA01469805.fits
Tranforming: SUPA01469806.fits
Tranforming: SUPA01469807.fits
Tranforming: SUPA01469808.fits
Tranforming: SUPA01469809.fits
Tranforming: SUPA01469810.fits
Tranforming: SUPA01469811.fits
Tranforming: SUPA01469812.fits
Tranforming: SUPA01469813.fits
Tranforming: SUPA01469814.fits
Tranforming: SUPA01469815.fits
Tranforming: SUPA01469816.fits
Tranforming: SUPA01469817.fits
Tranforming: SUPA01469818.fits
Tranforming: SUPA01469819.fits
Tranforming: SUPA01469820.fits
Tranforming: SUPA01469821.fits
Tranforming: SUPA01469822.fits
Tranforming: SUPA01469823.fits
Tranforming: SUPA01469824.fits
Tranforming: SUPA01469825.fits
Tranforming: SUPA01469826.fits
Tranforming: SUPA01469827.fits
Tranforming: SUPA01469828.fits
Tranforming: SUPA01469829.fits
Tranforming: SUPA01469830.fits
Tranforming: SUPA01469831.fits
Tranform

## Project Individual Projections

This sub-process takes about 12 minutes.

In [10]:
from MontagePy.main import mImgtbl

imgdir = dirs['registered_image']
raw_image_tbl = os.path.join(image_dir, 'raw_image.tbl')

rtn = mImgtbl(imgdir, raw_image_tbl)
rtn

{'status': '0', 'count': 50, 'badfits': 0, 'badwcs': 0}

In [11]:
from MontagePy.main import mMakeHdr, mProjExec, mAdd
hdrfile = os.path.join(image_dir, 'mosaic.hdr')
rtn = mMakeHdr(raw_image_tbl, hdrfile )
rtn

{'status': '0',
 'count': 50,
 'ncube': 0,
 'naxis1': 11275,
 'naxis2': 9228,
 'clon': 312.68929162276174,
 'clat': 44.352934494522735,
 'lonsize': 0.63264025,
 'latsize': 0.5177830800000001,
 'posang': 359.9765342250354,
 'lon1': 313.12956525873705,
 'lat1': 44.09306981547718,
 'lon2': 312.24872080175743,
 'lat2': 44.09332776928199,
 'lon3': 312.2451090258769,
 'lat3': 44.61109299405244,
 'lon4': 313.13377011381607,
 'lat4': 44.61083275120581}

In [12]:
try:
    shutil.rmtree(dirs['projected_image'])
except:
    pass
os.mkdir(dirs['projected_image'])
projdir = dirs['projected_image']
rtn = mProjExec(imgdir, raw_image_tbl, hdrfile, projdir=projdir, quickMode=True)
rtn

{'status': '0', 'count': 50, 'failed': 0, 'nooverlap': 0}

## Create the Final Mosaic

This takes about one minute.

In [13]:
projdir = dirs['projected_image']
pimage_tbl = os.path.join(image_dir, 'pimages.tbl')

rtn = mImgtbl(projdir, pimage_tbl )
print(f'mImgtbl returned: {rtn}')

#coadd into a temp file
tmp_out = os.path.join(image_dir, 'tmp_mosaic.fits')

rtn = mAdd(projdir, pimage_tbl,  hdrfile, tmp_out, coadd=1)
print(f'mAdd returned: {rtn}')

# convert to single precision
mosaic_fits = os.path.join(rootdir,  fits_out)
with fits.open(tmp_out) as f:
    img_hdr=f[0].header
    img_data = f[0].data.astype(np.float32)
phdu = fits.PrimaryHDU(data = img_data, header = img_hdr)
phdu.writeto(mosaic_fits, overwrite=True)

try:
    os.remove(tmp_out)
    os.remove(os.path.join(rootdir, filter_name, 'tmp_mosaic_area.fits'))
except:
    pass

mImgtbl returned: {'status': '0', 'count': 50, 'badfits': 0, 'badwcs': 0}
mAdd returned: {'status': '0', 'time': 47.0}
